<a href="https://colab.research.google.com/github/oaarnikoivu/dissertation/blob/master/Sem_Eval_2018_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
import collections

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


## Load data

In [0]:
def load_data(filename):
  data = pd.read_csv(filename, sep='\t')
  return data

In [0]:
file_path = '/content/drive/My Drive/datasets/'

train = load_data(file_path + '2018-E-c-En-train.txt')
val = load_data(file_path + '2018-E-c-En-dev.txt')
test = load_data(file_path + '2018-E-c-En-test-gold.txt')

In [5]:
train.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2017-En-21441,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,2017-En-31535,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,2017-En-21068,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,2017-En-31436,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,2017-En-22195,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


Lets create a list of all labels to predicts and also a 'none' label to see how many tweets have no labels.

In [6]:
class_names = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 
              'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

train['none'] = 1-train[class_names].max(axis=1)
train.describe()

,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust,none
count,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000,6838.000000
mean,0.372039,0.143024,0.380521,0.181632,0.362240,0.102369,0.290143,0.116262,0.293653,0.052793,0.052208,0.029833
std,0.483384,0.350123,0.485550,0.385569,0.480683,0.303155,0.453862,0.320562,0.455468,0.223637,0.222463,0.170140
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
len(train), len(test), len(val)

(6838, 3259, 886)

Since there are a few empty tweets, we will replace these with: 'unknown'. 

In [0]:
train['Tweet'].fillna("unknown", inplace=True)
test['Tweet'].fillna("unknown", inplace=True)
val['Tweet'].fillna("unknown", inplace=True)

In [0]:
train_text = train['Tweet']
test_text = test['Tweet']
val_text = val['Tweet']
all_text = pd.concat([train_text, test_text, val_text])

## Text preprocessing

In [10]:
train_text.head()

0    “Worry is a down payment on a problem you may ...
1    Whatever you decide to do make sure it makes y...
2    @Max_Kellerman  it also helps that the majorit...
3    Accept the challenges so that you can literall...
4    My roommate: it's okay that we can't spell bec...
Name: Tweet, dtype: object

In [0]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def preprocessor(text):
    text = text.lower()
    text = re.sub(' +', ' ', text)
    #text = re.sub('#', '', text) # remove hashtags
    text = REPLACE_BY_SPACE_RE.sub('', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = ' '.join([word for word in text.split() if word not in STOPWORDS])
    text = text.strip()
  
    return text

In [12]:
preprocessor(train_text[250])

'dont #afraid space #dreams #reality #dream #make'

In [0]:
train_text = train_text.apply(preprocessor)
test_text = test_text.apply(preprocessor)
val_text = val_text.apply(preprocessor)

In [14]:
train_text.head()

0    worry payment problem may never joyce meyer #m...
1               whatever decide make sure makes #happy
2    max_kellerman also helps majority nfl coaching...
3    accept challenges literally even feel exhilara...
4    roommate okay cant spell autocorrect #terrible...
Name: Tweet, dtype: object

## Transforming text to a vector

### TF-IDF

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
def tfidf_features(train, val, test):
  vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=2,
                               token_pattern='(\S+)')
  
  vectorizer.fit(all_text)

  train_features = vectorizer.transform(train)
  test_features = vectorizer.transform(test)
  val_features = vectorizer.transform(val)

  return vectorizer, train_features, val_features, test_features, vectorizer.vocabulary_

In [0]:
vect, train_features, val_features, test_features, vocab = tfidf_features(train_text, val_text, test_text)

In [161]:
vocab['alcohol']

6265

## Train classifier

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [0]:
def train_classifier(X_train, X_val, X_test, pred_text):

  #clf = LogisticRegression()
  #clf = KNeighborsClassifier(n_neighbors=5, metric='manhattan')
  clf = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                      hidden_layer_sizes=(5, 2), random_state=1, max_iter=800)
  predictions = []

  for class_name in class_names:
    print('\n... Processing {}'.format(class_name))

    train_target = train[class_name]
    test_target = test[class_name]
    val_target = val[class_name]

    clf.fit(X_train, train_target)

    # compute training accuracy
    y_pred_X_train = clf.predict(X_train)
    print('Training accuracy is {}'.format(accuracy_score(train_target, y_pred_X_train)))
    
    # compute validation accuracy
    y_pred_X_val = clf.predict(X_val)
    print('Validation accuracy is {}'.format(accuracy_score(val_target, y_pred_X_val)))

    # compute testing accuracy
    y_pred_X_test = clf.predict(X_test)
    print('Testing accuracy is {}'.format(accuracy_score(test_target, y_pred_X_test)))

    for t in pred_text:
      pred = clf.predict(vect.transform([t]))
      if pred:
        predictions.append(t + " - " + class_name)

  return (clf, predictions)

In [276]:
pred_text = ["depression sucks!", 
             "so happy to be home from ibiza", 
             "apologies for retweeting a tweet",
             "I think Robert can now rejoice in peace",
             "too much fun being interviewed by the talented and funny #datphan",
             "#selfish #horrible person",
             "trumps only concern is personal profit",
             "He is crazy, mentally sick and spews hatred!",
             "brighten up a gloomy day with spray tan",
             "I feel exhausted, I tried to do a good job but apparently God does not see me right.",
             "Will I ever NOT be tired again?",
             "She was obviously moved by the music.",
             "fuck small talk tell me about what age did you start disappointing your parents"]

(clf, preds) = train_classifier(train_features, val_features, test_features, pred_text)


... Processing anger
Training accuracy is 0.9780637613337233
Validation accuracy is 0.7528216704288939
Testing accuracy is 0.7606627799938631

... Processing anticipation
Training accuracy is 0.9900555718046212
Validation accuracy is 0.8160270880361173
Testing accuracy is 0.8266339367904265

... Processing disgust
Training accuracy is 0.9785024861070488
Validation accuracy is 0.6873589164785553
Testing accuracy is 0.7038969008898435

... Processing fear
Training accuracy is 0.993711611582334
Validation accuracy is 0.8589164785553047
Testing accuracy is 0.8570113531758208

... Processing joy
Training accuracy is 0.9972214097689382
Validation accuracy is 0.7381489841986456
Testing accuracy is 0.7453206505062903

... Processing love
Training accuracy is 0.9722140976893828
Validation accuracy is 0.8837471783295711
Testing accuracy is 0.8579318809450752

... Processing optimism
Training accuracy is 0.9850833577069319
Validation accuracy is 0.6941309255079007
Testing accuracy is 0.675974225

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Training accuracy is 0.9913717461245979
Validation accuracy is 0.845372460496614
Testing accuracy is 0.8505676587910402

... Processing sadness
Training accuracy is 0.9704591985960808
Validation accuracy is 0.7178329571106095
Testing accuracy is 0.6989874194538201

... Processing surprise
Training accuracy is 0.9472067856098274
Validation accuracy is 0.9604966139954854
Testing accuracy is 0.9478367597422522

... Processing trust
Training accuracy is 0.9688505410938871
Validation accuracy is 0.9334085778781038
Testing accuracy is 0.9340288432034366


In [277]:
for p in preds:
  print(p)

apologies for retweeting a tweet - anger
#selfish #horrible person - anger
She was obviously moved by the music. - anger
fuck small talk tell me about what age did you start disappointing your parents - anger
#selfish #horrible person - disgust
He is crazy, mentally sick and spews hatred! - disgust
I feel exhausted, I tried to do a good job but apparently God does not see me right. - disgust
She was obviously moved by the music. - disgust
fuck small talk tell me about what age did you start disappointing your parents - disgust
#selfish #horrible person - fear
He is crazy, mentally sick and spews hatred! - fear
She was obviously moved by the music. - fear
so happy to be home from ibiza - joy
I think Robert can now rejoice in peace - joy
too much fun being interviewed by the talented and funny #datphan - joy
brighten up a gloomy day with spray tan - joy
I feel exhausted, I tried to do a good job but apparently God does not see me right. - joy
Will I ever NOT be tired again? - joy
so happ